In [1]:
import pygame
import random
import numpy as np
import copy
from statistics import mode 
import time
from math import inf
import collections
import operator

pygame 2.0.1 (SDL 2.0.14, Python 3.8.3)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
class Square:
    def __init__(self, x, y, color, size, neighbors):
        self.x = x
        self.y = y
        self.color = color
        self.size = size
        self.neighbors = neighbors
        self.active = False
        self.bad_square = False

    def display(self, screen):
        if not self.bad_square:
            pygame.draw.rect(screen, self.color, (self.x, self.y, self.size, self.size), border_radius=0) # left, top, width, height
        else:
            pygame.draw.rect(screen, self.color, (self.x, self.y, self.size, self.size), border_radius=self.size//2)

In [3]:
# Calculate all the neighbors of a square
def getNeighbors(i,j,m,n):
    # central area
    if i > 0 and j > 0 and i < m and j < n:
        return np.asarray([[(i-1), j], [i, (j-1)], [(i+1), j], [i, (j+1)]])
    # sides, apart from the corners
    elif i <= 0 and j > 0 and i < m and j < n:
        return np.asarray([[i, (j-1)], [(i+1), j], [i, (j+1)]])
    elif i > 0 and j <= 0 and i < m and j < n:
        return np.asarray([[(i-1), j], [(i+1), j], [i, (j+1)]])
    elif i > 0 and j > 0 and i >= m and j < n:
        return np.asarray([[(i-1), j], [i, (j-1)], [i, (j+1)]])
    elif i > 0 and j > 0 and i < m and j >= n:
        return np.asarray([[(i-1), j], [i, (j-1)], [(i+1), j]])
    # corners
    elif i <= 0 and j <= 0 and i < m and j < n:
        return np.asarray([[(i+1), j], [i, (j+1)]])    
    elif i <= 0 and j > 0 and i < m and j >= n:
        return np.asarray([[i, (j-1)], [(i+1), j]])
    elif i > 0 and j <= 0 and i >= m and j < n:
        return np.asarray([[(i-1), j], [i, (j+1)]])
    else:
        return np.asarray([[(i-1), j], [i, (j-1)]])

In [4]:
# Select a square via a mouse click
def clickSquare(all_squares, x, y):
    for i in range(all_squares.shape[0]):
        for j in range(all_squares.shape[1]):
            # checking if the position of the mouse is inside the selected square
            if (all_squares[i][j].x < x and all_squares[i][j].y < y 
                    and (all_squares[i][j].x + all_squares[i][j].size) > x 
                            and (all_squares[i][j].y + all_squares[i][j].size) > y):
                return all_squares[i][j]
    return None

In [5]:
# change the color of the active square
def isValidNeighbor(all_squares, selected_square, active_color):
    if not selected_square.active:
        working_array = [] # array to check if the square clicked is further away but directly connected to the working area
        for el1,el2 in selected_square.neighbors:
            # check if the square clicked is next to the working area
            if all_squares[el1][el2].active:
                selected_square.active = True
                return selected_square.color, True
            # check if the neighboring square is the same color as the one clicked
            elif all_squares[el1][el2].color == selected_square.color:
                working_array.extend(all_squares[el1][el2].neighbors)
        # checking all the connected squares of the same color for connection to the working area
        while working_array:
            el = working_array.pop(0)
            if all_squares[el[0]][el[1]].color == selected_square.color:
                working_array.extend(all_squares[el[0]][el[1]].neighbors)
            elif all_squares[el[0]][el[1]].active:
                selected_square.active = True
                return selected_square.color, True
    return active_color, False

In [6]:
# Change the status of the adjoining squares into active and return the addition to the score
def activateSquares(square_rows, square_columns, all_squares, active_color, first_round=False):
    changed = 1
    bad_square = False
    any_changes = True
    while any_changes:
        any_changes = False
        for i in range(square_rows): 
            for j in range(square_columns): 
                if all_squares[i][j].active:
                    for el1,el2 in all_squares[i][j].neighbors:
                        if all_squares[el1][el2].color == active_color and not all_squares[el1][el2].active:
                            all_squares[el1][el2].active = True
                            any_changes = True
                            changed += 1              
                    if all_squares[i][j].bad_square and not first_round:
                        all_squares[i][j].bad_square = False
                        bad_square = True
                        any_changes = True
    # up to (square_rows+square_columns)//2 squares - normal counting
    # up to (square_rows+square_columns) squares -2x points
    # then 3x points etc.
    # if containts bad square, the points are halved
    tipping_point = (square_rows+square_columns)//2
    if not bad_square:
        return changed*((changed//tipping_point)+1)
    else:
        return changed//2

In [7]:
# Change the shape of the "bad" squares
def reshapeSquares(square_rows, square_columns, all_squares, active_color):
    any_changes = True
    while any_changes:
        any_changes = False
        for i in range(square_rows): 
            for j in range(square_columns): 
                if all_squares[i][j].bad_square:
                    for el1,el2 in all_squares[i][j].neighbors:
                        if all_squares[el1][el2].color == all_squares[i][j].color and not all_squares[el1][el2].bad_square:
                            all_squares[el1][el2].bad_square = True
                            any_changes = True

In [8]:
# Recolor all the active squares
def recolor(square_rows, square_columns, all_squares, active_color):                               
    for i in range(square_rows): 
        for j in range(square_columns): 
            if all_squares[i][j].active:
                all_squares[i][j].color = active_color

In [9]:
def updateScreen(square_rows, square_columns, all_squares, screen):
    for i in range(square_rows): 
        for j in range(square_columns): 
                all_squares[i][j].display(screen)

In [10]:
def checkFinished(square_rows, square_columns, all_squares):
    # checks if all squares are of the same color
    color = all_squares[0][0].color
    for i in range(square_rows):
        for j in range (square_columns):
            if all_squares[i][j].color != color:
                return False
    return True

In [11]:
# Update the text displayed
def updateText(width, height, score, moves, greedy_moves, screen):
    text_size = width//15
    if text_size < 27:
        font = pygame.font.Font(None, text_size) # (font, font_size)
    else:
        font = pygame.font.Font(None, 28) # (font, font_size)
    text = font.render('Score:' + str(score) + "    " + "Moves:" + str(moves) +"    " + "Maximum moves:" +str(greedy_moves), True, 'black') # (text, text_has_smooth_edges?, color)
    text_surface = text.get_rect(center=(width//2, height+20)) # text surface with the center point of the object (left, top)
    screen.fill('white', rect=text_surface, special_flags=0)
    screen.blit(text, text_surface)

In [12]:
def greedyApproach(square_rows, square_columns, all_squares):
    # determines minimal number of moves using greedy approach
    steps = 0
    while(not checkFinished(square_rows, square_columns, all_squares)):
        next_step = getMostImportantNeighbor(square_rows, square_columns, all_squares)
        recolor(square_rows, square_columns, all_squares, next_step)
        activateSquares(square_rows, square_columns, all_squares, next_step)
        steps += 1
    return steps

In [13]:
def getMostImportantNeighbor(square_rows, square_columns, all_squares):
    # get most often occuring color of neighboring squares from active area
    all_colours = list()
    for i in range(square_rows): 
        for j in range(square_columns): 
            current_square = all_squares[i][j]
            if current_square.active == True:
                for neighbor_i, neighbor_j in current_square.neighbors:
                    if all_squares[neighbor_i][neighbor_j].color != current_square.color:
                        all_colours.append(((neighbor_i, neighbor_j),all_squares[neighbor_i][neighbor_j].color))    
    return mode([x[1] for x in list(set(all_colours))])

In [14]:
def solver(max_steps, square_rows, square_columns, all_squares, algo=1, tipping_point=0.75, score=0):
    # determines the maximal possible score
    steps = 0
    while not checkFinished(square_rows, square_columns, all_squares):
        all_colors, active_bad, non_active_bad = getNeighboringColors(square_rows, square_columns, all_squares)
        next_color = decisionAlgorithm(square_rows, square_columns, all_colors, active_bad, non_active_bad, algo, tipping_point)
        recolor(square_rows, square_columns, all_squares, next_color)
        score += activateSquares(square_rows, square_columns, all_squares, next_color) 
        steps += 1
    if steps <= max_steps:
        return score
    else:
        return 0

In [15]:
def getNeighboringColors(square_rows, square_columns, all_squares):
    all_colours = list()
    active_bad_square_color = []
    non_active_bad_square_color = []
    for i in range(square_rows): 
        for j in range(square_columns): 
            current_square = all_squares[i][j]
            if current_square.active:
                for neighbor_i, neighbor_j in current_square.neighbors:
                    if all_squares[neighbor_i][neighbor_j].color != current_square.color:
                        all_colours.append(((neighbor_i, neighbor_j),all_squares[neighbor_i][neighbor_j].color))  
                        if all_squares[neighbor_i][neighbor_j].bad_square:
                            active_bad_square_color.append(((neighbor_i, neighbor_j),all_squares[neighbor_i][neighbor_j].color))
            elif current_square.bad_square:
                non_active_bad_square_color.append(((neighbor_i, neighbor_j),all_squares[neighbor_i][neighbor_j].color))
    return collections.Counter([x[1] for x in list(set(all_colours))]), [x[1] for x in list(set(active_bad_square_color))], [x[1] for x in list(set(non_active_bad_square_color))]

In [16]:
def decisionAlgorithm(square_rows, square_columns, all_colors, active_bad, non_active_bad, algo, tipping_point):
    
    modulo = (square_rows+square_columns)//2
    
    # algorithm 0
    # pick the algorithm used at random
    if algo == 0:
        algo = random.randint(1, 11)
    
    # algorithm 1
    # 1) prioritize the color of the bad square (if there)
            # chose the bad square with the smaller value
    # 2) don't, if both of the bad squares are the same color
    # 3) chose the maximal/second-maximal/etc. value based on the added score 
    # 0-75% 1x - choose, 75-100% 1x - wait, 0-75% 2x - choose, 75-100% 2x - wait, etc.   
    if algo == 1:
        if active_bad:
            if len(active_bad) == 1:
                if active_bad != non_active_bad:
                    return active_bad[0]
            else:
                if all_colors[active_bad[0]] < all_colors[active_bad[1]]:
                    return active_bad[0]
                else:
                    return active_bad[1]
        else:
            for key in all_colors:
                if all_colors[key]%modulo < (modulo*tipping_point):
                    return key        
        
    # algorithm 2
    # 1) prioritize the color of the bad square (if there)
            # chose the bad square with bigger value
    # 2) don't, if both of the bad squares are the same color
    # 3) chose the maximal/second-maximal/etc. value based on the added score 
    elif algo == 2:
        if active_bad:
            if len(active_bad) == 1:
                if active_bad != non_active_bad:
                    return active_bad[0]
            else:
                if all_colors[active_bad[0]] > all_colors[active_bad[1]]:
                    return active_bad[0]
                else:
                    return active_bad[1]        
        else:
            for key in all_colors:
                if all_colors[key]%modulo < (modulo*tipping_point):
                    return key 

    # algorithm 3
    # 3) chose the maximal/second-maximal/etc. value based on the added score 
    elif algo == 3:
        for key in all_colors:
            if all_colors[key]%modulo < (modulo*tipping_point):
                return key        
    
    return max(all_colors.items(), key=operator.itemgetter(1))[0]

In [17]:
def main():
    pygame.init()
    
    pygame.display.set_caption('Flood it')
    (width, height) = (400, 400)
    screen = pygame.display.set_mode((width, height+40))
    square_size = 25
    
    score = 0
    moves = 0
    
    blue = 'blue'
    cyan = 'cyan'
    limegreen = 'limegreen'    
    yellow = 'yellow'
    purple = 'darkviolet'
    black = 'black'
    white = 'white'
    colors = [blue, cyan, limegreen, yellow, purple]
    
    screen.fill(white)
    
    # Number of squares on the table
    (square_rows, square_columns) = int(width/square_size), int(height/square_size)
    all_squares = np.empty((square_rows, square_columns), dtype=object)
    
    # Random "good" square assigned
    random_x1, random_y1 = random.randint(0, square_rows-1), random.randint(0, square_rows-1)
    random_x2, random_y2 = random.randint(0, square_rows-1), random.randint(0, square_rows-1)
    
    for i in range(square_rows): # creating and coloring every row
        for j in range(square_columns): # creating and coloring every column
            if not ((i == random_x1 and j == random_y1) or ((i == random_x2 and j == random_y2))):
                all_squares[i][j] = Square(i*square_size, j*square_size, random.choice(colors), 
                                           square_size, getNeighbors(i, j, square_rows-1, square_columns-1))
            elif i == random_x1 and j == random_y1:
                all_squares[i][j] = Square(i*square_size, j*square_size, random.choice(colors), 
                                           square_size, getNeighbors(i, j, square_rows-1, square_columns-1))
                all_squares[random_x1][random_y1].bad_square = True
            else:
                all_squares[i][j] = Square(i*square_size, j*square_size, random.choice(colors), 
                                           square_size, getNeighbors(i, j, square_rows-1, square_columns-1))
                all_squares[random_x2][random_y2].bad_square = True
 
    # Make the upper left square active (and the adjoining ones of the same color)
    all_squares[0][0].active = True
    active_color = all_squares[0][0].color
    activateSquares(square_rows, square_columns, all_squares, active_color, True)   
   
    start_greedy = time.time()
    minimal_steps = greedyApproach(square_rows, square_columns, copy.deepcopy(all_squares))
    end_greedy = time.time()
    
    tipping_points = [0.5, 0.75, 0.8, 0.85, 0.9, 0.95, 1]
    solutions = {}
    for tipping_point in tipping_points:
        for i in range(0, 4):
            max_score = solver(minimal_steps, square_rows, square_columns, copy.deepcopy(all_squares), algo=i, tipping_point=tipping_point)
            solutions["algo_"+str(i)+"_"+str(tipping_point)] = max_score
            
    print("Best possible score is:", max(solutions.items(), key=operator.itemgetter(1))[1])

    reshapeSquares(square_rows, square_columns, all_squares, active_color)
    updateScreen(square_rows, square_columns, all_squares, screen)        
    updateText(width, height, score, moves, minimal_steps,  screen)
                                  
    # Changes need to be made before the flip() function is called.
    pygame.display.flip()    
    
    running = True
    while running:  
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                running = False
                
            if event.type == pygame.MOUSEBUTTONDOWN:
                (mouseX, mouseY) = pygame.mouse.get_pos()
                selected_square = clickSquare(all_squares, mouseX, mouseY)
                valid_move = False
                                
                if selected_square is not None:
                    active_color, valid_move = isValidNeighbor(all_squares, selected_square, active_color)
                    
                if valid_move:
                    moves += 1
                    recolor(square_rows, square_columns, all_squares, active_color)  
                    score += activateSquares(square_rows, square_columns, all_squares, active_color)
                    updateScreen(square_rows, square_columns, all_squares, screen)            
                    updateText(width, height, score, moves, minimal_steps,  screen)
                
                if checkFinished(square_rows, square_columns, all_squares) and moves <= minimal_steps:
                    print("You win!")
                if moves > minimal_steps:
                    print("You lose!")
                    
                # TODO : error handle clicking the wrong titles
                    
                pygame.display.flip()    
    
    pygame.quit()

In [18]:
if __name__=="__main__":
    main()

Best possible score is: 361
You win!
